In [ ]:
import cache
import meta
import meta_session

import nept
import numpy as np
import pandas as pd
import scipy.stats
import scipy.signal
from scipy.ndimage import median_filter
from shapely.geometry import LineString, Point
from shapely.ops import split
import statsmodels.api as sm
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

from utils import save_ttest_results

%matplotlib inline

In [ ]:
info = meta_session.r066d1
group = "all"

def get(key, info=None, group=None):
    if info is not None:
        return cache.load(f"ind-{info.session_id}", key)
    return cache.load(f"grp-{group}", key)

In [ ]:
trials = get("trials", info=info)
mostly_shortcut_idx = get("mostly_shortcut_idx", group=group)
trial_proportions_bytrial = get("trial_proportions_bytrial", group=group)